# Advent of Code 2022, Day 8: Treetop Tree House

See https://adventofcode.com/2022/day/8 for details.



In [1]:
import numpy as np

Step 1: A not-so-optimal attempt, which is improved later

In [4]:
# Check which trees are visible
# Strategy: for each row, we move L->R, storing the largest number. 

def minimum_visibility( tree_array ):
    dims = tree_array.shape
    lookleft = np.full(shape=dims, fill_value = -1)
    lookright = np.full(shape=dims, fill_value = -1)
    lookup = np.full(shape=dims, fill_value = -1)
    lookdown = np.full(shape=dims, fill_value = -1)

    for x in range(dims[0]):
        lowleft = 0
        lowright = 0
        for y in range(dims[1]):
            lookleft[x,y] = lowleft
            lookright[x,-y-1] = lowright
            lowleft = max(lowleft, tree_array[x,y])
            lowright = max(lowright, tree_array[x,-y-1])

    for y in range(dims[1]):
        lowup = 0
        lowdown = 0
        for x in range(dims[0]):
            lookup[x,y] = lowup
            lookdown[-x-1,y] = lowdown
            lowup = max(lowup, tree_array[x,y])
            lowdown = max(lowdown, tree_array[-x-1,y])

    # Write down the lowest a tree that would be invisible at location x,y, 
    # by taking the minimum of all three look matrices
    minvis = np.full(shape=dims, fill_value = -1)
    for x in range(dims[0]):
        for y in range(dims[1]):
            minvis[x,y] = min( [lookup[x,y], lookdown[x,y], lookleft[x,y], lookright[x,y] ] )

    return (minvis)

def number_of_visible_trees( minvis, tree_array ):
    # now check if the given height is HIGHER than minvis:
    dims = tree_array.shape

    trees_visible = np.full(shape=dims, fill_value = -1)
    for x in range(dims[0]):
        for y in range(dims[1]):
            trees_visible[x,y] = ( minvis[x,y] == 0 or minvis[x,y] < tree_array[x,y] )

    num_vis =  np.count_nonzero(trees_visible)

    return(num_vis, trees_visible)


In [5]:
# Reading the data: 
f = open('input8test.txt')
tree_array = []
for row in f:
    row = row.strip('\n')
    row = [ int(j) for j in row ]
    tree_array.append(list(row))
tree_array = np.array( tree_array )


minvis = minimum_visibility( tree_array )
(num_vis, trees_visible) = number_of_visible_trees( minvis, tree_array )
print('number of visble trees', num_vis)
print("invisible: ", tree_array.shape[0] * tree_array.shape[1] - num_vis )

    

number of visble trees 21
invisible:  4


Step 2: a much better attempt to solve the first challenge.

In [6]:
# Method 2: we simply do a brute-force counting from each position. 
# Reading the data: 

def check_if_visible( x, y, tree_array, dims ):
    if x == 0 or x == dims[0]-1 : 
        return 1
    if y == 0 or y == dims[1]-1 :
        return 1
    
    # check on top
    #print("checking TOP ", x, ' ', y, ' ', tree_array[x,y], ' ', tree_array[0:x, y] )
    if np.max( tree_array[0:x, y] ) < tree_array[x,y] :
        return 1
    # check on bottom
    if np.max( tree_array[x+1:, y] ) < tree_array[x,y] :
        return 1
    # check the left
    if np.max( tree_array[x, 0:y] ) < tree_array[x,y] :
        return 1
    # check the right
    #print("checking RIGHT ", x, ' ', y, ' ',  tree_array[x,y], ' ',  tree_array[x, y+1:] )
    if np.max( tree_array[x, y+1:] ) < tree_array[x,y] :
        return 1

    return 0
    


f = open('input8.txt')
tree_array = []
for row in f:
    row = row.strip('\n')
    row = [ int(j) for j in row ]
    tree_array.append(list(row))
tree_array = np.array( tree_array )


tree_array

array([[2, 2, 2, ..., 0, 4, 0],
       [1, 1, 0, ..., 0, 0, 1],
       [1, 3, 0, ..., 0, 3, 3],
       ...,
       [0, 3, 0, ..., 0, 0, 2],
       [4, 0, 4, ..., 4, 3, 0],
       [3, 2, 4, ..., 1, 3, 2]])

In [7]:
dims = tree_array.shape
counter = 0
for x in range(dims[0]):
    for y in range(dims[1]):
        if check_if_visible( x, y, tree_array, dims ):
            counter += 1 

print(counter)


1533


## Part 2: Viewing distance

In [9]:
def view_distance_in_list( lst, current ):
    # determine viewing distance, given a lst of trees, assuming viewer is at height 'current'
    counter = 0
    for j in lst: 
        counter += 1
        if current <= j :
            break
    return(counter)



def get_scenic_score( x, y, tree_array ):
    dims = tree_array.shape
    current = tree_array[x,y]
    if x == 0 or x == dims[0]-1 : 
        return 0
    if y == 0 or y == dims[1]-1 :
        return 0

    # check on top
    toplist = np.flip( tree_array[0:x, y] )
    topscore = view_distance_in_list( toplist, current )
    #print(topscore)

    # check on bottom
    botlist = tree_array[x+1:, y]
    botscore = view_distance_in_list( botlist, current )
    #print(botscore)

    # check the left
    leftlist = np.flip( tree_array[x, 0:y] )
    leftscore = view_distance_in_list( leftlist, current )

    # check the right
    rightlist =  tree_array[x, y+1:]
    rightscore = view_distance_in_list( rightlist, current )

    return topscore * botscore * leftscore * rightscore

def get_max_scenic_score( tree_array ):
    dims = tree_array.shape
    max_found = -1
    for x in range(dims[0]):
        for y in range(dims[1]):
            score = get_scenic_score( x, y, tree_array)
            if score > max_found :
                max_found = score

    return max_found
    


f = open('input8.txt')
tree_array = []
for row in f:
    row = row.strip('\n')
    row = [ int(j) for j in row ]
    tree_array.append(list(row))
tree_array = np.array( tree_array )

print("best scenic score: ", get_max_scenic_score(tree_array) )

x = 3
y = 2
print( tree_array[x+1:, y] )
print( tree_array[x,y])
get_scenic_score( 3,2, tree_array )

best scenic score:  345744
[1 4 2 5 3 4 5 0 5 5 2 4 5 1 0 4 4 0 0 4 2 0 3 1 3 5 0 2 4 2 1 6 2 6 3 4 4
 7 2 2 7 1 5 5 7 0 4 6 0 5 5 4 4 4 2 3 1 1 0 5 4 7 2 3 5 2 5 6 1 4 2 4 4 4
 0 0 4 2 0 4 4 5 4 1 2 0 3 3 1 4 2 1 0 4 4]
4


30